# Exploring Impresso Tools (NER, NEL, article embeddings)  


This notebook provides an overview and demonstration of the core tools integrated into the Impresso application, focusing on components that power text understanding, entity recognition, and retrieval capabilities across the corpus. 

This notebook documents production-level tools that are permanent within the Impresso infrastructure.

Specifically, we cover three major components:

* **Named Entity Recognition (NER)** – identifying and classifying named entities (e.g., people, places, organizations) in historical newspaper text using the [impresso-project/ner-stacked-bert-multilingual](https://huggingface.co/impresso-project/ner-stacked-bert-multilingual) model.
* **Named Entity Linking (NEL)** – resolving recognized entities to canonical entries in knowledge bases such as Wikidata, using the [impresso-project/nel-mgenre-multilingual](https://huggingface.co/impresso-project/nel-mgenre-multilingual) model.
* **Article Embeddings** – generating embeddings of full articles using [gte-multilingual-base](https://huggingface.co/Alibaba-NLP/gte-multilingual-base) to enable semantic search with:
    - **In-corpus queries** – selecting a query directly from the *Impresso* corpus.  
    - **Out-of-corpus queries** – embedding an external query (e.g., manually formulated or from another source).  


In [ ]:
from impresso import connect

impresso = connect()

### Named entity recognition


In [2]:
text = """
    Jean-Baptiste Nicolas Robert Schuman ( 
    29 June 1886 – 4 September 1963) was a Luxembourg-born French 
    statesman. Schuman was a Christian democratic (Popular 
    Republican Movement) political thinker and activist. 
    """
result = impresso.tools.ner(
    text=text
)
result

,type,surfaceForm,function,name,confidence.ner,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName
id,,,,,,,,,
2:41:pers:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,91.25,2,41,N/A,N/A
46:80:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,29 June 1886 – 4 September 1963,N/A,N/A,77.90,46,80,N/A,N/A
88:98:org:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,org,Luxembourg,N/A,N/A,25.12,88,98,N/A,N/A


### Named entity linking

For the system to know what entity to link, we need to surround it with the markers [START] and [END]. Leave spaces between the entity and the markers.

In [3]:
text = """
    [START] Jean-Baptiste Nicolas Robert Schuman [END] ( 
    29 June 1886 – 4 September 1963) was a Luxembourg-born French 
    statesman. Schuman was a Christian democratic (Popular 
    Republican Movement) political thinker and activist. 
    """

result = impresso.tools.nel(
    text=text,
)
result

,type,confidence.nel,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,
,unk,99.93,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman


### Named entity recognition and linking

This method will do entity recognition and linking at the same time.

In [4]:
text = """
    Jean-Baptiste Nicolas Robert Schuman ( 
    29 June 1886 – 4 September 1963) was a Luxembourg-born French 
    statesman. Schuman was a Christian democratic (Popular 
    Republican Movement) political thinker and activist. 
    """
result = impresso.tools.ner_nel(
    text=text,
)
result

,type,surfaceForm,function,name,confidence.ner,confidence.nel,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,,,,,,,
2:41:pers:nel-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,91.25,96.76,2,41,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
46:80:time:nel-mgenre-multilingual,time,29 June 1886 – 4 September 1963,N/A,N/A,77.90,86.46,46,80,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
88:98:org:nel-mgenre-multilingual,org,Luxembourg,N/A,N/A,25.12,100.00,88,98,Q32,Luxembourg,https://en.wikipedia.org/wiki/Luxembourg


### Article embeddings

All content items in the Impresso data of type _article_ longer than 800 characters were embedded using [gte-multilingual-base](https://huggingface.co/Alibaba-NLP/gte-multilingual-base) which is the latest in the GTE (General Text Embedding) family that has a strong multilingual capability.

In [2]:
text = """
    Jean-Baptiste Nicolas Robert Schuman ( 
    29 June 1886 – 4 September 1963) was a Luxembourg-born French 
    statesman. Schuman was a Christian democratic (Popular 
    Republican Movement) political thinker and activist. 
    """

text_space_embedding = impresso.tools.embed_text(text=text, target="text")
text_space_embedding

'gte-768:Fao1vWPioj3Msv+7LwqfPLicTb2kPgO+pkpAPVhbkL1cdi891oY9vYv1dDzSHTC9dJ0MvYGTZT3d87C90kt6PZ81gT0pg4c7x6XRPdfOC70xllU9WJrxO/pBwDwDI1C8YveXvZ3gyrzDjek9FkkhvXFAoL2LjYo9W0XwvKLdFLsg10C7QGhjPcAUHD1Z4YC8yFZhPbxhnToLYza9cFl8vMkdYTw0DZG8Mr10vX8zIjo/ZTs8AYC1PGyAn70CFaS7PAyNvNqxPT0AUyC9eiAyPXVJo7y1OGM9r/8TPdFwez1HIaS9lXUHvGb4Aj4N9oK9wTlNvQBmOrtQG9E7WqmPvUjIYLx6jZa8hOEFPV6vkLzIEoM9latiPej5i7s72xI8NmLBurlZuDzdEyw8gD7mPcUoQLytEUk6CfLAPfnsOD1UKy29vVunvBrRozzb8jU9op9OPJnZqTyJeE+8bchbvT3uhLxL2WU9toKuPfffBrwuxZk8H2PLvG6TWby+ei09OKuLvAUJTL0mvh08oOuDvYTpiTsBNQ47CNTAPB3frbzvPi88CUQ9vcza/LwKvRs91lUCPPlTCjx/bH+9dEkovK43K70N5bC8+BMxvcgJBz1vxoo9JQygvHvAkz3VDry8nWWzuwuSJ72RL+a8lJP5vJPTur1O3uG8Ph27vGo/LL0q7ew8mvK9vBMQmD3kaIU93Lg1PejcIb1ffAg9y4wuvDBo2LxlEzc9SWfWunvsKb228fy8F7iKPBLTSL2OFiO9T4+JPOr5jzuas5K8grgMuy6EQzzGCF69W2DZPKX7JT3C/908EIxYvaKCMrwC3e28KIJJvLBgGj0P+Im8LMJuvGClrT2C4W67DOx6u/g0rbummEI8UquLPKSzvrwj1nk8OyIoPXR3ILxpzxy98WElPW2ejDxP2jK8vsGBvKCdBD1QD/K8sieVPc6Cjr0mm6k7U8SCvEEc9LyS4Iq9STFGvGc7uLsB0bg

#### Search by text embedding with `tools.embed_text` with an out-of-corpus embedding

The `include_embeddings` flag is `False` by default to avoid downloading embeddings for items that may not be needed.

In [5]:
result = impresso.search.find(
  embedding=text_space_embedding,
  limit=5,
  include_embeddings=True,
)

result

,copyrightStatus,type,sourceMedium,title,topics,embeddings,transcriptLength,totalPages,languageCode,isOnFrontPage,...,pageNumbers,collectionUids,entities.locations,entities.persons,entities.organisations,entities.newsAgencies,mentions.locations,mentions.persons,mentions.organisations,mentions.newsAgencies
uid,,,,,,,,,,,,,,,,,,,,,
IMP-1949-01-20-a-i0001,in_cpy,ar,print,Les voyages de M. Robert Schuman,"[{'uid': 'tm-fr-all-v2.0_tp29_fr', 'relevance'...",[gte-768:byoMvc6wYT2x8rA6qQcxPUqpML2TUoq9aprOP...,643,1,fr,True,...,[1],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EXP-1958-03-11-a-i0148,in_cpy,ar,print,Robert Schuman parle de la zone de libre-échange,"[{'uid': 'tm-fr-all-v2.0_tp00_fr', 'relevance'...",[gte-768:LcFFvDLFOD3U6ZW8d+9EPfC3Er3GYJq9GsIGP...,657,1,fr,False,...,[6],[],"[{'uid': '2-54-Berne', 'count': 3}, {'uid': '2...","[{'uid': '2-50-Robert_Schuman', 'count': 5}]","[{'uid': '2-53-États_pontificaux', 'count': 2}]",[],"[{'surfaceForm': 'BERNE', 'mentionConfidence':...","[{'surfaceForm': 'M. Robert Schuman, ancien pr...",[{'surfaceForm': 'Association romande et de l'...,[]
luxland-1963-09-13-a-i0008,in_cpy,ar,print,Lettre à l'éditeur,"[{'uid': 'tm-fr-all-v2.0_tp45_fr', 'relevance'...",[gte-768:KBsevTg+oj0D76y82ZYQPXHxhr3uGrW9TWa9P...,481,1,fr,False,...,[4],[],"[{'uid': '2-54-France', 'count': 1}, {'uid': '...","[{'uid': '2-50-Robert_Schuman', 'count': 2}]","[{'uid': '2-53-Institut_de_France', 'count': 1}]",[],"[{'surfaceForm': 'France', 'mentionConfidence'...",[{'surfaceForm': 'professeurs à la Faculté de ...,"[{'surfaceForm': 'Institut de France', 'mentio...",[]


Get embeddings by ID.

In [9]:
first_item_embeddings = impresso.content_items.get_embeddings(result.df.index[0])[0]
first_item_embeddings

'gte-768:byoMvc6wYT2x8rA6qQcxPUqpML2TUoq9aprOPHVB1Lxfdgm8ohwxvfk6R7sODvi7grSrOtsjuT1UZ6G9F0pvPRxrKz0MONu7vAyrPYfLczv7g9g8oyPQPccqGD1FGwG7fpNvvWb4X7w0bp09EN8DvtWGBL6mxoU8H5qlvQeW7Du35ZY80d/bPTRunT2VLJq8BMBPPfmtOz2CtCu8oq0vvWQUBT2f0x+9knAUvWLHALyhNkg8whmoO5YKnbz6HD29jc6lPEO6vTuJuUK9O+1HPWm8yzxfBfo7/7LSPP7MaTzSybe9VO0NPWTYLT5bR4+8sfIwu4v4CLtvt5e7W9QavC1+47zD96o8hesLPRbnxru4WIs9RLZKPXeob70u5ww8H6D9uxKmmzviCka9li/GPSh5zLwkMLs8ZKGQPTOBlTwhfBu9purnPIoc6zzmsKc8Te5OPFIVY7wNFt685U/kvCaJmLymTx49ZKdoPavlszvUkRY9sXfWvB+wSTyPQ/88pfLNvM8Gk72ZU668EE2+vPZkKr24ajw8NWMLPRmVDr3CFTW9BbYEPCWterzjawk91LX4vJF6XzzE6KW9fgLxvEJTIr13Daa8Y50dvdOogT1g+64990oTvJL1uT3YSSk9vXc5ux4zCr103qu8BFVBvakLpL2Iwai70tWQuwwQhr27q2c9gNYovbhJhj1qH/Q8lMNwPf1PKr2hNsg8RRsBvcvIEzxnWSM9OCOEvENHybxydxC9DKPpPH6Nl725wxk9rhghvdWGhD0NidK8JDihui9SmzwC9P07UgsYugAUFj1Naak8/sIevVtZwDubvjw8s10/vWdZIzy5UKW8+waZuzSLYD1QP8a7XTfDPAtScryY6B+6n2p2OqQM5bs6bJW72UU2PXEs8TruIM28UC0VPfjH0jzX7Fi7Tk8SvOvFCjxdJZI73d8+PLb/Lb3TQJ+860owu3lenb1DKoa9hlaau/HCOz2lbSg

#### Search by text embedding with `tools.embed_text` with an in-corpus embedding

In [11]:
result = impresso.search.find(
  country="FR",
  embedding=first_item_embeddings,
  limit=2
)

result

,copyrightStatus,type,sourceMedium,topics,transcriptLength,totalPages,languageCode,isOnFrontPage,publicationDate,issueUid,...,collectionUids,entities.locations,entities.persons,entities.organisations,entities.newsAgencies,mentions.locations,mentions.persons,mentions.organisations,mentions.newsAgencies,title
uid,,,,,,,,,,,,,,,,,,,,,
oecaen-1941-05-15-a-i0018,in_cpy,ar,print,"[{'uid': 'tm-fr-all-v2.0_tp29_fr', 'relevance'...",263,1,fr,False,1941-05-15T00:00:00+00:00,oecaen-1941-05-15-a,...,[],"[{'uid': '2-54-Europe', 'count': 1}, {'uid': '...",[],[],[],"[{'surfaceForm': 'Europe', 'mentionConfidence'...",[],[],[],NaN
lepetitparisien-1926-09-18-a-i0054,in_cpy,ar,print,"[{'uid': 'tm-fr-all-v2.0_tp03_fr', 'relevance'...",232,1,fr,False,1926-09-18T00:00:00+00:00,lepetitparisien-1926-09-18-a,...,[],"[{'uid': '2-54-Paris', 'count': 1}, {'uid': '2...","[{'uid': '2-50-Austen_Chamberlain', 'count': 1}]","[{'uid': '2-53-France', 'count': 1}]","[{'uid': '4-55-Havas', 'count': 4}, {'uid': ''...","[{'surfaceForm': 'Genève', 'mentionConfidence'...","[{'surfaceForm': 'BRIAND', 'mentionConfidence'...","[{'surfaceForm': 'France', 'mentionConfidence'...","[{'surfaceForm': 'Havas', 'mentionConfidence':...",L'ENTREVUE BRIAND-STRESEMANN


### Other tools

#### Convert embedding to an array of floats and back

In [12]:
original_embedding = impresso.content_items.get_embeddings("lepetitparisien-1928-05-07-a-i0047")[0]
original_embedding

'gte-768:8gU7vdWAjz3oWoK9o8o+PUHElL3dkT+9zky+vFKAfT34Ejg9S8Y/vSqUE71SgSE98ArSvMs/AT4x0a68X4zWPXoJnD2/+Rs7OJ/IPS4awLz4VjI9nRS0PedVKz3oMy69kP8Qve/ujzyU9L49KpSTvVwUS73ilTI9rkowvTgcaz11fhg9Q++bPUEzlj3ZLRC8UXdzPQKYwDw1x4C7CEn0PBl2pju5K4K886TavJmmFr2Zg/U8IMFiOy66g7zq/dS8QUG3PFI0Hb3fHSe8PIE+PYrQljy7ZKq7piDNPNNeUjwkbuO9Byf3PFWAPT7+w2u9pQNnvHniR7tFl0W8Qh86PDd9S7wGjW482+mVPCM6krw0AY09J6xiPSQXcb1gatm8fbX4O6JNnLtH0ZE7IT8pPccBAr0wFSm8x5KAPUfy6jyCKRE8ep5NPIPCdTwWsHI9EkzDOu4x5rySWja8q1RevCF9aDzkX1k9uSpePNbW3btIQBM9ez1tu+t7GzypaDo993ivvLvEZr37rEC8jUf+u0lq9rrQheY8MXBOu07ejjq+G5k8fracvACdV7wuWP88zKQUvQ230TgY6r69iz8Yvc7dPL0YyEG9AtejPJBRrD22msM9f3F+vN3CgbwTRyw9HO6xvPSfQ7wQSJC91Ll3vJaOx73uMWa8VKyoPPD8sL3Mc9I8ndonvanXOz12+zo9es8PPTSqmrs7NLo83IL6vIi4x7w7lZo9tXSTPCwf1zywO6u9S0mdPAsFur2/GvU6GJd/vALXIz3BVMG8C95lvDzAIT0o60U8oAqGvRgbATzG43c9hMOZvDc/jDrCjg27o8q+vOm/FTz0YQQ7CYPAvDFwTj3K2ck6g8L1PFTrC73zgt06L7nfPF7fFT0zjbS7OfuRPBJMwzu6R8S8PY/fPH702zw2Q/+7cDvCvDyBPj0W4TS7TKTCPGjSe70+6yg85+apvPWHtL37bV297hANPVw2yD0g8iS

In [13]:
from impresso.util.embeddings import embedding_to_vector, vector_to_embedding

array = embedding_to_vector(original_embedding)
array[:10]  # show first 10 values

[-0.04565996676683426,
 0.07006994634866714,
 -0.06364995241165161,
 0.04657996818423271,
 -0.07263994961977005,
 -0.046769965440034866,
 -0.02322998270392418,
 0.061889953911304474,
 0.04493996500968933,
 -0.04681996628642082]

In [14]:
recreated_embedding = vector_to_embedding(array, 'gte-768')
recreated_embedding

'gte-768:8gU7vdWAjz3oWoK9o8o+PUHElL3dkT+9zky+vFKAfT34Ejg9S8Y/vSqUE71SgSE98ArSvMs/AT4x0a68X4zWPXoJnD2/+Rs7OJ/IPS4awLz4VjI9nRS0PedVKz3oMy69kP8Qve/ujzyU9L49KpSTvVwUS73ilTI9rkowvTgcaz11fhg9Q++bPUEzlj3ZLRC8UXdzPQKYwDw1x4C7CEn0PBl2pju5K4K886TavJmmFr2Zg/U8IMFiOy66g7zq/dS8QUG3PFI0Hb3fHSe8PIE+PYrQljy7ZKq7piDNPNNeUjwkbuO9Byf3PFWAPT7+w2u9pQNnvHniR7tFl0W8Qh86PDd9S7wGjW482+mVPCM6krw0AY09J6xiPSQXcb1gatm8fbX4O6JNnLtH0ZE7IT8pPccBAr0wFSm8x5KAPUfy6jyCKRE8ep5NPIPCdTwWsHI9EkzDOu4x5rySWja8q1RevCF9aDzkX1k9uSpePNbW3btIQBM9ez1tu+t7GzypaDo993ivvLvEZr37rEC8jUf+u0lq9rrQheY8MXBOu07ejjq+G5k8fracvACdV7wuWP88zKQUvQ230TgY6r69iz8Yvc7dPL0YyEG9AtejPJBRrD22msM9f3F+vN3CgbwTRyw9HO6xvPSfQ7wQSJC91Ll3vJaOx73uMWa8VKyoPPD8sL3Mc9I8ndonvanXOz12+zo9es8PPTSqmrs7NLo83IL6vIi4x7w7lZo9tXSTPCwf1zywO6u9S0mdPAsFur2/GvU6GJd/vALXIz3BVMG8C95lvDzAIT0o60U8oAqGvRgbATzG43c9hMOZvDc/jDrCjg27o8q+vOm/FTz0YQQ7CYPAvDFwTj3K2ck6g8L1PFTrC73zgt06L7nfPF7fFT0zjbS7OfuRPBJMwzu6R8S8PY/fPH702zw2Q/+7cDvCvDyBPj0W4TS7TKTCPGjSe70+6yg85+apvPWHtL37bV297hANPVw2yD0g8iS

In [15]:
f"Original and recreated embeddings are the same: {original_embedding == recreated_embedding}"

'Original and recreated embeddings are the same: True'

### Embed an image

In [16]:
impresso.tools.embed_image("https://impresso-project.ch/assets/images/posts/rep-thomas.png", target="multimodal")

'openclip-768:/aKPPipmmT7ziZ0/+jaWPqb4lb5up0w/pYmevSLlvz5p4TW9XbhBv3uSO7+PATE+6IP3Ppwugb4e6cU+GDAwvxzPLD/ddM4+vOPyPkfXhD4O6kW+sNy4PRitcD9NsP8+4Marvfd2zL4ao5W+qgGaPomXBT/XNxG/RN8LP++jhr8U/kK/WCjjPuqsPj+5nfm+byGPvpGSL7/JmJ69ReE1P1Ymojypa9W+iN6/v2qMAD7EYRu/IBDDPVoOFL/YSHW/jljfPrBmMj9kQOu+duvivl4Scb8Q120/bMMxPtwLvL63tu49xDiJPHekFT/hLCW/4a5iv9TBhr482H++Jj6NPkLkhr6+XiQ/w2caveqZmD4wGQG/JKOxPpYZCz//JVa/mGMoP7CVrb6SMbe9cssZPwXSuD/3NGK/RRvYvWeeib4r7Po9FlfsPUAHJT/MZ6w/HI+fPs7yL780EJK+Kf3tvgUtxb60Gdo+HmIfPvAg7Tx6sL4+lku2P00+xT5fF8k+1ABAvuLUlj6H+Vg/L8TvvrxA1b4dnkw/T+oAPwV4Bz+ndBG/iPGrPzAgsL6tjve+bxvhPcivr78RMXe/fqrzvgRYtz5J6CW/aij8vQPKgb57ewu//I75vfD2UT44VLK+0iEdPlo4yz8HQXs/uqeUP2zWDT65m2G/MNTjvUzhCD/bZuk+gFh0PuFsabzAHSC+ewifPwikgz5iPzc+Dq+dvTRjlT/yI8M+PpK2v2/8Ib3bteG+eYIzvSzmjL/+GjG/tCofvi7gdD7c5am+RpGsPj6il77sJtM+LaZOQDlDGb5bUxk/2UIuvyxn4r40VVK/Upc6vrTzzb5WPo09nRiGPfuqLr8BhSu9dh4nP2rVgD8biU+/jMJVPsO5o741cUS+C+UNPVC+Yr9D4uC+WpTUvTRwE79uMDa/WWbDvUaAP7+uC2W+J834PeCKCr8shTA+xgJjv+qEGr/fRJW++CoYv+ybjz